## The Lambda Retreat - Day 5


Lazy Evaluation Metacircular Interpreter

In [ ]:
%%html
<style type="text/css">
div.CodeMirror span.CodeMirror-matchingbracket {
    color: #FF0000;
    font-weight: bold;
}
</style>

In [ ]:




;; %%file lazy2.scm
(import "builtins")
(define python builtins)

(define (eval exp env)
;;   (python.print "eval" exp (repr-env env))
  (cond ((self-evaluating? exp) 
         exp)
        ((variable? exp) 
         (lookup-variable-value exp env))
        ((quoted? exp)
         (text-of-quotation exp))
        ((definition? exp)
         (eval-definition exp env))
        ((if? exp)
         (eval-if exp env))        
        ((begin? exp)
         (eval (begin->lambda exp) env))
        ((let? exp)
         (display (format "~a \n" (let->lambda exp)))
         (eval (let->lambda exp) env))
        ((let*? exp)
         (display (format "~a \n" (let*->let exp)))
         (eval (let*->let exp) env))
        ((lambda? exp)
         (make-procedure
          (lambda-parameters exp)
          (lambda-body exp)
          env))
        ((application? exp)
         ;; @@@ CHANGE 1 - 
         (xapply (actual-value (operator exp) env)
                 (operands exp)
                 env))
        (else 
         (error "EVAL - unknown expression type" exp))))
        
(define (xapply proc arguments env)
  (cond ((primitive-procedure? proc)
         (apply-primitive-procedure 
          proc 
          ;; CHANGE 2
          (list-of-arg-values arguments env)))
        ((compound-procedure? proc)
         (eval-sequence
          (procedure-body proc)
          (extend-env
           (procedure-parameters proc)
           ;; CHANGE 3
           (list-of-delayed-args
            arguments
            env)
           (procedure-env proc))))
        (else
         (error "APPLY - unknown procedure type" proc))))

; predicates
(define (self-evaluating? e) (number? e))
(define (variable? e) (symbol? e))
(define (quoted? e) (tagged-list? e 'quote ))
(define (definition? e) (tagged-list? e 'define))
(define (if? e) (tagged-list? e 'if))
(define (lambda? e) (tagged-list? e 'lambda))
(define (begin? e) (tagged-list? e 'begin))
(define (let? e) (tagged-list? e 'let))
(define (let*? e) (tagged-list? e 'let*))
(define (application? e) (pair? e))


(define (primitive-procedure? e) (tagged-list? e 'primitive))
(define (compound-procedure? e) (tagged-list? e 'compound-procedure))

(define (tagged-list? exp tag)
  (and (pair? exp) 
       (eq? (car exp) tag)))
; quote

(define (text-of-quotation e)
  (cadr e))

;; define

(define (eval-definition exp env)
  (add-binding-to-frame! 
   (definition-variable exp)
   (delay-it (definition-value exp) env)
   (top-frame env)))

(define (definition-variable exp) (cadr exp))
(define (definition-value exp) (caddr exp))

;; if

(define (eval-if exp env)
  ;; CHANGE 5
  (if (actual-value (if-predicate exp) env)
      (eval (if-consequent exp) env)
      (eval (if-alternative exp) env)))

(define (if-predicate exp) (cadr exp))
(define (if-consequent exp) (caddr exp))
(define (if-alternative exp) (cadddr exp))

;; apply

(define (operator exp) (car exp))
(define (operands exp) (cdr exp))

(define (apply-primitive-procedure proc args)
  (apply (primitive-procedure-value proc) args))

(define (primitive-procedure-value proc) (cadr proc))

  
;; CHANGE 4
(define (list-of-arg-values exps env)
  (map (lambda (exp) 
         (actual-value exp env))
       exps))
  
(define (list-of-delayed-args exps env)
  (map (lambda (exp) (delay-it exp env))
       exps))
  
(define (delay-it exp env)
  (list 'promise exp env))
  
(define (promise? obj) 
  (tagged-list? obj 'promise))
(define (promise-exp obj) (cadr obj))
(define (promise-env obj) (caddr obj))
  
(define (actual-value exp env)
  (force-it (eval exp env)))

(define (force-it obj)
  (if (promise? obj)
      (actual-value (promise-exp obj)
                    (promise-env obj))
      obj))
  
;; lambda

(define (lambda-parameters exp) (cadr exp))
(define (lambda-body exp) (cddr exp))

(define (make-procedure params body env)
  (list 'compound-procedure params body env))

(define (procedure-parameters proc) (cadr proc))
(define (procedure-body proc) (caddr proc))
(define (procedure-env proc) (cadddr proc))

(define (begin->lambda exp)
  (list (cons 'lambda 
              (cons '() 
                    (begin-body exp)))))

(define (begin-body exp)
  (cdr exp))

(define (let->lambda exp) 
      (cons (cons 'lambda 
                   (cons (letvar exp) (letbody exp))) 
            (letargs exp)))

(define (letvar exp) 
      (map (lambda (x) (car x)) (letparams exp)))

(define (letargs exp) 
      (map (lambda (x) (cadr x)) (letparams exp)))

(define (letbody exp) 
      (caddr exp))

; let*

(define letparams cadr)

(define (dropfirstletparam exp) 
  (list 'let* (cdr (letparams exp)) (letbody exp)))

(define (let*->let exp) 
      (let ((params (letparams exp)))
            (if (null? params)
                (letbody exp)
                (list
                  'let
                  (list (car params))
                  (let*->let (dropfirstletparam exp))))))


; frame

(define (make-frame vars vals)
  (cons vars vals))

(define frame-vars car)
(define frame-vals cdr)

(define (lookup-frame var frame)
  (define (scan vars vals)
    (cond [(null? vars) '<notfound>]
          [(eq? (car vars) var)
           (car vals)]
          [else (scan (cdr vars) (cdr vals))]))
  (scan (frame-vars frame)
        (frame-vals frame)))

(define (add-binding-to-frame! var val frame)
  (define (append-var!)
      (set-car! frame (cons var (car frame)))
      (set-cdr! frame (cons val (cdr frame))))
  (define (scan vars vals)
    (cond ((null? vars) (append-var!))
          ((eq? (car vars) var) (set-car! vals val))
          (else (scan (cdr vars) (cdr vals)))))
  (scan (car frame) (cdr frame)))

;; eval sequence

(define (eval-sequence exps env)
  (if (null? (cdr exps))
      (eval (car exps) env)
      (begin
       (eval (car exps) env)
       (eval-sequence (cdr exps) env))))


; env

(define primitives
  (list
   (list 'x 10)
   (list '+ +)
   (list '- -)   
   (list '* *)
   (list '= =)
   (list '> >)
   (list '< <)
   (list 'display display)
   (list 'newline newline)   
   (list 'print python.print)))

(define primitive-variables (map car primitives))
(define primitive-values 
  (map (lambda (p) 
         (list 'primitive (cadr p))) 
       primitives))

(define initial-env '())
(define (setup-env)
  (cons (make-frame 
         primitive-variables
         primitive-values)
        initial-env))

(define (top-frame env) (car env))
(define (enclosing-env env) (cdr env))
(define empty-env? null?)


(define (extend-env vars vals env)
  (let [(frame (make-frame vars vals))]
    (cons frame env)))

(define (lookup-variable-value var env)
;;   (python.print "lookup-variable-value" var (repr-env env))
  (if (empty-env? env)
      (error "Unbound variable" (symbol->string var))
      (let [(frame (top-frame env))]
        (let [(val (lookup-frame var frame))]
          (if (eq? val '<notfound>)
              (lookup-variable-value var (enclosing-env env))
              val)))))
      
(define (list-of-values exps env)
    (if (null? exps)
        '()
        (let ((first (eval (car exps) env)))
          (let ((rest (list-of-values 
                       (cdr exps) env)))
            (cons first rest)))))
     
(define env (setup-env))
(define (try-eval expr)
  (python.print ">" expr)
  (let ((val (eval expr env)))
    (if (not (eq? val '<void>))
        (python.print (force-it val)))))

(define (repr-env env)
  (map repr-frame env))

(define (repr-frame frame)
  (car frame))



;; (begin->lambda '(begin a b))





In [ ]:

(try-eval '1)
;; (try-eval 'x)
;; (try-eval '(quote x))
;; (try-eval '(define a 10))
;; (try-eval 'a)
;; (try-eval '(if #t 10 20))
;; (try-eval '(if #f 10 20))
(try-eval '(+ 4 5))
;; (try-eval '(lambda () 4))
;; (try-eval '(lambda (x) (+ x 1)))
;; (try-eval '(define f (lambda (x) (+ x 1))))
;; (try-eval '(f 4))
(try-eval '(define fact (lambda (n) (if (= n 0) 1 (* n (fact (- n 1)))))))
(try-eval '(fact 5))
;; (try-eval '(define peep (lambda (x) (print x) x)))

;;(try-eval '(+ (peep 2) (peep 3) (peep 4)))

;; (try-eval '(define new-if 
;;              (lambda 
;;                (p c a) 
;;                (if p c a))))
;; (try-eval '(new-if #t 10 20))
;; (try-eval '(new-if #f 10 20))
;; (try-eval '(new-if #t 10 bad-var))

;; (try-eval '(define fact 
;;              (lambda (n) 
;;                (if (= n 0) 
;;                    1 
;;                    (* n (fact (- n 1)))))))
;; (try-eval '(fact 5))

(try-eval '(define x 10))
(try-eval '(define y x))
(try-eval '(define x 20))
(try-eval 'y)

(try-eval '(define ones (cons 1 ones)))
(try-eval '(define take (lambda (n seq)
                          (if (= n 0)
                              '()
                              (begin 
                               (display (car seq))
                               (newline)
                               (take (- n 1) (cdr seq)))))))

(try-eval '(define cons 
             (lambda (p q)
               (lambda (m) (m p q)))))

(try-eval '(define car 
             (lambda (pair) 
               (pair (lambda (p q) p)))))

(try-eval '(define cdr 
             (lambda (pair) 
               (pair (lambda (p q) q)))))

(try-eval '(define x (cons 1 2)))
(try-eval '(car x))
(try-eval '(cdr x))

;; (try-eval '(define (let params body) 
;;                  (lambda ())))

;; (try-eval '(take 2 
;;                  (cons 1 
;;                        (cons 2 
;;                              (cons 3 
;;                                    (cons 4 5))))))

;; (try-eval '(take 10 ones))


(load "lazy2.scm")

In [ ]:
(try-eval '(let ((x 1)) (x)))

In [ ]:
(try-eval '(let ((x 1) (y 2)) (+ x y)))

In [ ]:
(try-eval '(let* ((x 1) (y (+ x 1))) (+ x y)))

```
(let* ((x 1)
      (y (+ x 1)))
  (+ x y))


((lambda (x y) (+ x y)) 1 2)
---

(let ((x 1))
  (let ((y (+ x 1)))
    (+ x y)))


((lambda (x y) (+ x y)) 1 2)

(let*->let exp) 
```